In [1]:
from datetime import datetime, timedelta, timezone
import json
from heapq import heappush, heappop
import requests
import itertools

In [4]:
# API-ключ для доступа к Яндекс.Расписаниям
API_KEY = "95e48825-709d-4296-952b-eed993562fd5"

# Функция для получения списка всех городов и их кодов
def get_cities_mapping():
    url = "https://api.rasp.yandex.net/v3.0/stations_list/"
    params = {
        "apikey": API_KEY,
        "format": "json",
        "lang": "ru_RU"
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        cities_mapping = {}

        # Проходим по всем странам, регионам и населенным пунктам
        for country in data.get("countries", []):
            for region in country.get("regions", []):
                for settlement in region.get("settlements", []):
                    city_name = settlement.get("title")
                    city_code = settlement.get("codes", {}).get("yandex_code")
                    if city_name and city_code:
                        cities_mapping[city_name] = city_code

        return cities_mapping
    return None

# Функция для сохранения сопоставления городов и их кодов в JSON-файл
def save_cities_mapping(cities_mapping, filename="cities_mapping.json"):
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(cities_mapping, file, indent=4, ensure_ascii=False)
    print(f"Сопоставление городов и их кодов сохранено в {filename}.")

# Основной код
if __name__ == "__main__":
    # Получаем список всех городов и их кодов
    cities_mapping = get_cities_mapping()
    # Сохраняем сопоставление городов и их кодов в JSON-файл
    save_cities_mapping(cities_mapping)

Сопоставление городов и их кодов сохранено в cities_mapping.json.


In [6]:
transport_translation = {
    "plane": "самолет",
    "train": "поезд",
    "suburban": "электричка",
    "bus": "автобус",
    "water": "водный транспорт",
    "helicopter": "вертолет"
}


def format_time(iso_time):
    if not iso_time:
        return "Нет данных"
    try:
        dt = datetime.fromisoformat(iso_time)
        return dt.strftime("%d %B %Y, %H:%M")
    except ValueError:
        return iso_time


def format_duration(seconds):
    hours = int(seconds // 3600)  # Получаем количество полных часов
    remainder = int(seconds % 3600)  # Остаток секунд после вычисления часов
    minutes = int(remainder // 60)  # Получаем количество полных минут
    return f"{hours} часов {minutes} минут"


def get_schedule_between_cities(from_city_code, to_city_code, date):
    url_search = "https://api.rasp.yandex.net/v3.0/search/"
    params = {
        "apikey": "95e48825-709d-4296-952b-eed993562fd5",
        "lang": "ru_RU",
        "format": "json",
        "from": from_city_code,
        "to": to_city_code,
        "date": date,
        "transfers": True
    }
    response = requests.get(url_search, params=params)
    return response.json() if response.status_code == 200 else []
def process_schedule(graph, from_city, to_city, schedule):
    if not schedule:
        return

    if from_city not in graph:
        graph[from_city] = {}
    if to_city not in graph[from_city]:
        graph[from_city][to_city] = []

    for route in schedule.get('segments', []):
        route_info = {
            'departure': datetime.fromisoformat(route['departure']).astimezone(timezone.utc),
            'arrival': datetime.fromisoformat(route['arrival']).astimezone(timezone.utc),
            'duration': (datetime.fromisoformat(route['arrival']) - datetime.fromisoformat(
                route['departure'])).total_seconds(),
            'segments': []
        }

        if route.get('has_transfers', False):
            for detail in route.get('details', []):
                if 'thread' in detail:
                    segment = {
                        'from_city': detail['from']['title'],
                        'from_station': detail['from'].get('station', {}).get('title', detail['from']['title']),
                        'to_city': detail['to']['title'],
                        'to_station': detail['to'].get('station', {}).get('title', detail['to']['title']),
                        'departure': datetime.fromisoformat(detail['departure']).astimezone(timezone.utc),
                        'arrival': datetime.fromisoformat(detail['arrival']).astimezone(timezone.utc),
                        'transport_type': detail['thread']['transport_type'],
                        'price': detail.get('tickets_info', {}).get('places', [{}])[0].get('price', {}).get('whole', 0)
                    }
                    route_info['segments'].append(segment)
        else:
            tickets_info = route.get('tickets_info', {})
            places = tickets_info.get('places', [{}])
            price = places[0].get('price', {}).get('whole', 0) if places else 0

            segment = {
                'from_city': route['from']['title'],
                'from_station': route['from'].get('station', {}).get('title', route['from']['title']),
                'to_city': route['to']['title'],
                'to_station': route['to'].get('station', {}).get('title', route['to']['title']),
                'departure': datetime.fromisoformat(route['departure']).astimezone(timezone.utc),
                'arrival': datetime.fromisoformat(route['arrival']).astimezone(timezone.utc),
                'transport_type': route['thread']['transport_type'],
                'price': price  # Используем исправленное значение цены
            }
            route_info['segments'].append(segment)

        graph[from_city][to_city].append(route_info)

def is_connection_possible(prev_arrival, next_departure):
    min_transfer_time = timedelta(minutes=30)
    return (next_departure - prev_arrival) >= min_transfer_time


def time_aware_dijkstra(graph, start, end, start_time):
    heap = []
    counter = itertools.count()  # Уникальный счетчик для сравнения
    heappush(heap, (0, next(counter), start, start_time, []))
    visited = set()

    while heap:
        total_time, _, current_city, current_time, path = heappop(heap)

        if current_city == end:
            return path

        if (current_city, current_time) in visited:
            continue
        visited.add((current_city, current_time))

        for neighbor in graph.get(current_city, {}):
            for route in graph[current_city][neighbor]:
                if route['departure'] < current_time:
                    continue

                if not path or is_connection_possible(path[-1]['arrival'], route['departure']):
                    new_time = route['arrival']
                    new_total = total_time + (new_time - current_time).total_seconds()
                    new_path = path + [route]

                    heappush(heap, (new_total, next(counter), neighbor, new_time, new_path))

    return None


def find_best_route(graph, start, city1, city2, start_time):
    path1 = time_aware_dijkstra(graph, start, city1, start_time)
    if not path1:
        return None

    path2 = time_aware_dijkstra(graph, city1, city2, path1[-1]['arrival'])
    if not path2:
        return None

    return path1 + path2
#Функция для вывода деталей маршрута
def print_route_details(route):
    print("Детали маршрута:")
    total_duration_seconds = 0  # Общее время в пути в секундах
    total_transfer_seconds = 0  # Общее время ожидания на пересадках в секундах
    previous_arrival = None

    for segment in route:
        for detail in segment['segments']:
            if previous_arrival:
                transfer_time = detail['departure'] - previous_arrival
                if transfer_time > timedelta(minutes=14):
                    print(f"  Пересадка в {detail['from_city']} ({detail['from_station']})")
                    print(f"    Время ожидания: {format_duration(transfer_time.total_seconds())}")
                    print()
                total_transfer_seconds += transfer_time.total_seconds()

            print(f"  {detail['from_city']} ({detail['from_station']}) -> {detail['to_city']} ({detail['to_station']})")
            print(f"    Тип транспорта: {transport_translation.get(detail['transport_type'], 'неизвестен')}")
            print(f"    Время отправления: {detail['departure'].strftime('%d %B %Y, %H:%M')}")
            print(f"    Время прибытия: {detail['arrival'].strftime('%d %B %Y, %H:%M')}")
            duration = detail['arrival'] - detail['departure']
            print(f"    Время в пути: {format_duration(duration.total_seconds())}")
            print(f"    Стоимость билета: {detail['price']} RUB" if detail['price'] else "    Стоимость билета: Не указана")
            print()

            previous_arrival = detail['arrival']
            total_duration_seconds += duration.total_seconds()

    # Общее время в пути = время в пути всех сегментов + время ожидания на пересадках
    total_time_seconds = total_duration_seconds + total_transfer_seconds
    print(f"Общее время в пути: {format_duration(total_time_seconds)}")

if __name__ == "__main__":
    with open("cities_mapping.json", "r", encoding="utf-8") as file:
        cities_mapping = json.load(file)

    start_city_name = input("Введите название города, откуда начнется путешествие: ")
    city1_name = input("Введите название города, который хотите посетить: ")
    city2_name = input("Введите название города, который хотите посетить: ")
    date = input("Введите дату в формате YYYY-MM-DD: ")

    start_city = cities_mapping.get(start_city_name)
    city1 = cities_mapping.get(city1_name)
    city2 = cities_mapping.get(city2_name)

    if not all([start_city, city1, city2]):
        print("Ошибка ввода городов.")
    else:
        graph = {}
        print("Мы работаем над вашим маршрутом...")

        for from_city, to_city in [(start_city, city1), (city1, start_city),
                                   (start_city, city2), (city2, start_city),
                                   (city1, city2), (city2, city1)]:
            schedule = get_schedule_between_cities(from_city, to_city, date)
            process_schedule(graph, from_city, to_city, schedule)

        start_time = datetime.fromisoformat(f"{date}T00:00:00+00:00").astimezone(timezone.utc)
        best_route = find_best_route(graph, start_city, city1, city2, start_time)

        if best_route:
            print(f"Лучший маршрут: {start_city_name} -> {city1_name} -> {city2_name}")
            print_route_details(best_route)
        else:
            print("Не удалось найти подходящий маршрут.")
            print("Не удалось найти подходящий маршрут.")


Введите название города, откуда начнется путешествие: Владивосток
Введите название города, который хотите посетить: Киров
Введите название города, который хотите посетить: Ульяновск
Введите дату в формате YYYY-MM-DD: 2025-03-28
Мы работаем над вашим маршрутом...
Лучший маршрут: Владивосток -> Киров -> Ульяновск
Детали маршрута:
  Владивосток (Владивосток) -> Москва (Москва)
    Тип транспорта: самолет
    Время отправления: 28 March 2025, 03:50
    Время прибытия: 28 March 2025, 12:45
    Время в пути: 8 часов 55 минут
    Стоимость билета: Не указана

  Пересадка в Москва (Москва)
    Время ожидания: 4 часов 5 минут

  Москва (Москва) -> Киров (Киров)
    Тип транспорта: самолет
    Время отправления: 28 March 2025, 16:50
    Время прибытия: 28 March 2025, 18:35
    Время в пути: 1 часов 45 минут
    Стоимость билета: Не указана

  Пересадка в Киров (Киров)
    Время ожидания: 0 часов 25 минут

  Киров (Киров) -> Москва (Москва)
    Тип транспорта: самолет
    Время отправления: 28 Ma